In [1]:
import requests
import pickle
import yfinance as yf
import numpy as np
from sqlalchemy import create_engine
import pandas as pd
import pandas_market_calendars as mcal
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import time
import torch
from torch.utils.data import DataLoader
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset

In [2]:
class Calendar:
    def __init__(self, start_date, end_date):
        # 取得美股（NYSE）交易日曆
        nyse = mcal.get_calendar('NYSE')
        # 獲取該時間範圍內的交易日
        schedule = nyse.valid_days(start_date=start_date, end_date=end_date)
        # 轉為 DataFrame
        trading_days = schedule.tz_convert(None).date
        self.trading_days = [str(trading_day) 
                             for trading_day in trading_days]

    def count_trading_days(self, start, end):
        # 計算 start ~ end 之間的交易日數量
        return sum(1 for day in pd.date_range(start, end) if day.date() in self.trading_days)

In [3]:
class OptionQuery:
    def __init__(self,ticker):
        self.price = yf.download(ticker, start='2010-01-01', end='2025-02-01', progress=False)
        self.ticker = ticker
        
    def _get_close_price(self, DATE):
        return self.price.loc[DATE]['Close']

    def _get_raw_option_df(self, DATE):
        with sqlite3.connect("options_data.db") as conn:
            cursor = conn.cursor()
        
            # 執行查詢
            cursor.execute("SELECT * FROM options_data WHERE date = ?", (DATE,))
            
            # 取得所有符合條件的資料
            rows = cursor.fetchall()
            # 取得欄位名稱
            columns = [desc[0] for desc in cursor.description]
        
        # 轉成 DataFrame 方便顯示
        return pd.DataFrame(rows, columns=columns)

    def _get_normalized_option_data(self, DATE):
   
        
        df = self._get_raw_option_df(DATE)
        

        
        if df.empty:
            return None
            
        # 計算剩餘天數
        
        df = df[df['volume']!=0]

        if df.empty:
            return None
        
        df["expiration"] = pd.to_datetime(df["expiration"])
        df["date"] = pd.to_datetime(df["date"])
        # 計算交易日數
        df["days_to_expiration"] = (df["expiration"] - df["date"]).dt.days / 365
        # 刪除不需要欄位
        df = df.drop(columns=["contractID", "date", "expiration", "symbol"])

        
        
        df = pd.get_dummies(df, columns=["type"])#.astype(int)
        df.columns = df.columns.str.lower()
        
        
        df[["type_call", "type_put"]] = df[["type_call", "type_put"]].astype(int)
        #df[["type_CALL", "type_PUT"]] = df[["type_Call", "type_Put"]].astype(int)

        
        
        df['open_interest'] =  df['open_interest'] / df['open_interest'].sum() *100
        df['bid_size'] = df['bid_size'] / df['bid_size'].sum() * 100
        df['ask_size'] = df['ask_size'] / df['ask_size'].sum() * 100
        
        df['volume'] = df['volume'] / df['volume'].sum() * 100
        df[['last', 'mark', 'bid', 'ask']] = df[['last', 'mark', 'bid', 'ask']].div(df['strike'], axis=0)
        close_price = self._get_close_price(DATE)
        df['strike'] =  df['strike'] / close_price
        
        #print(df)

        # ✅ 檢查是否有 NaN 值
        if df.isna().any().any():
            return None

        tensor = self.Z_score(torch.tensor(df.values))
        #print(pd.DataFrame(tensor))
        return tensor

    def Z_score(self, tensor, execute=True):
        if not execute:
            return tensor
        else:
            # 🔹 只標準化前 15 列（第 0~14 列）
            cols_to_normalize = list(range(15))  # 選擇前 15 列
            mean = tensor[:, cols_to_normalize].mean(dim=0, keepdim=True)  # 計算列均值
            std = tensor[:, cols_to_normalize].std(dim=0, keepdim=True)  # 計算列標準差
            
            # **避免除以 0，確保數值穩定**
            std[std == 0] = 1e-8
            
            # 🔹 標準化前 15 列，不影響其他列
            tensor[:, cols_to_normalize] = (tensor[:, cols_to_normalize] - mean) / std
            return tensor
            
    
        

    def option_data(self, DATE):
        return self._get_normalized_option_data(DATE)





In [4]:
query = OptionQuery('AAPL')
e = query.option_data('2015-01-06')

e

tensor([[-1.5654,  2.3158,  2.2752,  ...,  0.0082,  1.0000,  0.0000],
        [-1.1101,  1.3082,  1.2321,  ...,  0.0082,  1.0000,  0.0000],
        [-1.1101, -0.5697, -0.5682,  ...,  0.0082,  0.0000,  1.0000],
        ...,
        [ 2.7598, -0.3952, -0.3987,  ...,  2.0411,  1.0000,  0.0000],
        [ 2.9874, -0.4149, -0.4221,  ...,  2.0411,  1.0000,  0.0000],
        [ 3.2150, -0.4445, -0.4404,  ...,  2.0411,  1.0000,  0.0000]],
       dtype=torch.float64)

In [43]:
class OptionDataset:
    """
    透過 `OptionQuery` 動態加載選擇權數據
    """
    def __init__(self, ticker, start_date, end_date):
        self.query = OptionQuery(ticker)
        self.trading_days = Calendar(start_date, end_date).trading_days
        self.historical_return = self._get_historical_option_return(start_date, end_date)

        #print(len(self.trading_days))
        #print(len(self.historical_return))

    def _get_historical_option_return(self, start, end):
        historical_data = yf.download('AAPL', start='2014-01-01', end='2025-02-01', progress=False)
        historical_data["daily_return"] = historical_data["Adj Close"].pct_change()
        
        historical_return = historical_data['daily_return']
        historical_return = historical_return.shift(-1)[start : end]
        return historical_return
        
        
    def __len__(self):
        return len(self.trading_days)

    def __getitem__(self, idx):
        
        date = self.trading_days[idx]
        data = self.query.option_data(date)

        if data is None:
            data = torch.zeros((1, 18))

        feature = data.to(torch.float32)
        label = torch.tensor(self.historical_return.loc[date], dtype=torch.float32)
        return feature, label
#-----------------------------------------------------------------------------------------------------------------------------

In [51]:
# 建立訓練集 (TRAIN) 和 測試集 (TEST)
train_dataset = OptionDataset('AAPL', "2015-01-01", "2015-12-31")
test_dataset = OptionDataset('AAPL', "2022-01-01", "2022-12-31")
import torch
from torch.nn.utils.rnn import pad_sequence
import random

def my_collate_fn(batch):
    """
    自定義 collate_fn，填充時隨機挑選不重複的原始元素來補齊長度
    參數：
      batch: list, 每個元素是一個 tuple (feature, label)
             其中 feature 的形狀為 [Ni, d_dim]，label 為 scalar tensor
    返回：
      padded_features: tensor, 形狀為 [batch_size, N_max, d_dim]
      labels: tensor, 形狀為 [batch_size]
    """
    # 解析 batch 中的 features 和 labels
    features, labels = zip(*batch)
    
    # 找到 batch 內最大序列長度 (N_max)
    N_max = max(f.shape[0] for f in features)

    # 自定義填充：隨機從原數據挑選不重複的值來填充
    def pad_with_random_selection(tensor, target_length):
        current_length = tensor.shape[0]
        if current_length == target_length:
            return tensor  # 長度已經夠了，不需要填充
        
        # 取得原始數據
        original_data = tensor.tolist()
        while len(original_data) < target_length:
            # 隨機選取原始數據中的值，確保不重複
            sampled_values = random.sample(tensor.tolist(), min(len(tensor), target_length - len(original_data)))
            original_data.extend(sampled_values)
        
        # 轉回 tensor
        return torch.tensor(original_data, dtype=tensor.dtype, device=tensor.device)

    # 對所有 features 進行填充
    padded_features = [pad_with_random_selection(f, N_max) for f in features]
    
    # 堆疊成 batch tensor
    padded_features = torch.stack(padded_features, dim=0)  # [batch_size, N_max, d_dim]

    # 轉換 labels 為 tensor
    labels = torch.stack(labels, dim=0)  # [batch_size]

    return padded_features, labels*100


#def my_collate_fn(batch):
#    """
#    參數：
#      batch: list, 每個元素是一個 tuple (feature, label)
#             其中 feature 的形狀為 [Ni, d_dim]，label 為 scalar tensor
#    返回：
#      padded_features: tensor, 形狀為 [batch_size, N_max, d_dim]
#      labels: tensor, 形狀為 [batch_size]
#    """
#    # 將 batch 中的 feature 與 label 分離
#    features, labels = zip(*batch)
#    
#    # 對 feature 進行填充 (batch_first=True 表示輸出形狀為 [batch, seq_len, feature_dim])
#    padded_features = pad_sequence(features, batch_first=True, padding_value=0)
#    
#    # 將 label 組成 tensor
#    labels = torch.stack(labels, dim=0)
#    
#    return padded_features, labels


train_loader = DataLoader(train_dataset, batch_size=40, shuffle=True, collate_fn=my_collate_fn)
test_loader  = DataLoader(test_dataset, batch_size=40, shuffle=True, collate_fn=my_collate_fn)

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

# ---------------------------
# 2) Set Transformer 模組 (無 Mask 版本)
# ---------------------------

class MAB(nn.Module):
    """
    MAB(Q, K) = Multi-Head Attention Block
    這裡不實作 mask，因為一次只處理一個 day，不需要 padding。
    """
    def __init__(self, dim_Q, dim_K, dim_out, num_heads, ln=True):
        super().__init__()
        self.num_heads = num_heads
        self.dim_out = dim_out

        self.W_Q = nn.Linear(dim_Q, dim_out)
        self.W_K = nn.Linear(dim_K, dim_out)
        self.W_V = nn.Linear(dim_K, dim_out)

        self.fc = nn.Sequential(
            nn.Linear(dim_out, dim_out),
            nn.ReLU(),
            nn.Linear(dim_out, dim_out),
        )
        self.ln1 = nn.LayerNorm(dim_out) if ln else nn.Identity()
        self.ln2 = nn.LayerNorm(dim_out) if ln else nn.Identity()

    def forward(self, Q, K):
        """
        Q, K shape = (1, Nq, d_in), (1, Nk, d_in)
        這裡因為一次一天，所以 batch_size=1
        但也可以寫成 (batch, Nq, d_in) (batch, Nk, d_in) 只是 batch=1
        """
        B, Nq, _ = Q.shape
        _, Nk, _ = K.shape

        # 線性投影
        Q_ = self.W_Q(Q)  # (1, Nq, dim_out)
        K_ = self.W_K(K)  # (1, Nk, dim_out)
        V_ = self.W_V(K)  # (1, Nk, dim_out)

        d = self.dim_out
        d_head = d // self.num_heads
        # 拆多頭
        Q_ = Q_.view(B, Nq, self.num_heads, d_head).transpose(1, 2)  # (1, h, Nq, d_head)
        K_ = K_.view(B, Nk, self.num_heads, d_head).transpose(1, 2)
        V_ = V_.view(B, Nk, self.num_heads, d_head).transpose(1, 2)

        # 注意力 scores: (1, h, Nq, Nk)
        scores = torch.matmul(Q_, K_.transpose(-2, -1)) / (d_head**0.5)
        attn = torch.softmax(scores, dim=-1)
        H = torch.matmul(attn, V_)  # (1, h, Nq, d_head)

        # 拼回 (1, Nq, dim_out)
        H = H.transpose(1, 2).contiguous().view(B, Nq, d)

        # 殘差 + LayerNorm
        H = self.ln1(H + self.W_Q(Q))
        # 前饋
        H2 = self.fc(H)
        H = self.ln2(H + H2)
        return H

class SAB(nn.Module):
    """ Self-Attention Block: SAB(X) = MAB(X, X) """
    def __init__(self, dim_in, dim_out, num_heads=4, ln=True):
        super().__init__()
        self.mab = MAB(dim_in, dim_in, dim_out, num_heads=num_heads, ln=ln)

    def forward(self, X):
        # X shape = (1, N, dim_in)
        return self.mab(X, X)

class PMA(nn.Module):
    """
    Pooling by Multihead Attention:
    PMA(S, X) = MAB(S, X)
    num_seeds=1 -> 取得整個集合的一個向量表示
    """
    def __init__(self, dim_in, num_heads=4, num_seeds=1, ln=True):
        super().__init__()
        self.num_seeds = num_seeds
        self.dim_in = dim_in
        self.S = nn.Parameter(torch.Tensor(num_seeds, dim_in))
        nn.init.xavier_uniform_(self.S)
        self.mab = MAB(dim_in, dim_in, dim_in, num_heads=num_heads, ln=ln)

    def forward(self, X):
        # X shape = (1, N, dim_in)
        # S shape = (num_seeds, dim_in) -> (1, num_seeds, dim_in)
        B = X.shape[0]  # 一般情況 batch=1
        S = self.S.unsqueeze(0).expand(B, self.num_seeds, self.dim_in)
        H = self.mab(S, X)  # (1, num_seeds, dim_in)
        return H

class SetTransformer(nn.Module):
    """
    結合 SAB + PMA，最後用 Linear 做回歸
    """
    def __init__(self, dim_input=7, hidden_dim=128, num_heads=4):
        super().__init__()
        self.sab1 = SAB(dim_input, hidden_dim, num_heads=num_heads, ln=True)
        self.sab2 = SAB(hidden_dim, hidden_dim, num_heads=num_heads, ln=True)
        self.pma = PMA(hidden_dim, num_heads=num_heads, num_seeds=1, ln=True)
        self.fc = nn.Linear(hidden_dim, 10)  # 輸出 1 維(回歸)

    def forward(self, X):
        """
        X shape = (1, N, dim_input)
        """
        H = self.sab1(X)        # (1, N, hidden_dim)

        H = self.sab2(H)        # (1, N, hidden_dim)
        H = self.pma(H)         # (1, 1, hidden_dim)
        H = H.squeeze(1)        # (1, hidden_dim)
        out = self.fc(H)        # (1, 1)
        return out



In [53]:







class FC(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(18,80)
        self.fc2 = nn.Linear(80,160)
        self.fc3 = nn.Linear(160,200)
        self.fc4 = nn.Linear(200,256)
        
        self.fc = nn.Linear(256,30)
        
        self.LeakyReLU = nn.LeakyReLU()

    def Standardize(self, data):
        # 計算每個 batch & 每個 feature 在 200 個合約上的均值 & 標準差
        mean = data.mean(dim=1, keepdim=True)  # (3, 1, 18)，每個 batch 的 feature-wise 均值
        std =  data.std(dim=1, keepdim=True)    # (3, 1, 18)，每個 batch 的 feature-wise 標準差
        # 標準化，每個 feature 在 200 個合約上變成 0 均值、1 標準差
        normalized_output = (data - mean) / (std + 1e-6)  # 避免除以 0
        return normalized_output
        
    def forward(self, x):
        x = self.LeakyReLU(self.Standardize(self.fc1(x)))
        x = self.LeakyReLU(self.Standardize(self.fc2(x)))
        x = self.LeakyReLU(self.Standardize(self.fc3(x)))
        x = self.LeakyReLU(self.Standardize(self.fc4(x)))
        x = x.mean(dim=1)
        x = self.fc(x)

        return torch.tanh(x)/10*100
        
Input = torch.randn((3,100,18))
model = FC()
print(model(Input).shape)
        

torch.Size([3, 30])


In [54]:
1

1

In [55]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# 建立模型
#model = SetTransformer(dim_input=18, hidden_dim=128, num_heads=4)
model = FC()
model = model.to(device)


In [56]:
import torch.optim.lr_scheduler as lr_scheduler

# 初始化 Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

# 🔹 使用 `ReduceLROnPlateau`，當 `valid_loss` 停滯 5 個 Epoch，學習率縮小 0.5 倍
#scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

criterion = nn.MSELoss()
EPOCHS = 1000

In [57]:

def train_single_day():
    for epoch in range(EPOCHS):
        # **訓練階段**
        model.train()
        total_train_loss = 0.0

        count = 0
        
        correctness_list = []
        for batch in train_loader:
            count+=1
            if count%20==0:
                print(count)
                #print(total_train_loss)
            
            feats, label = batch
            #print(label.shape)
            label = label.unsqueeze(1).repeat(1,30)
            

            
            feats, label = feats.to(device), label.to(device)

            
                
            
                
            optimizer.zero_grad()
            preds = model(feats)  

            #print(preds.shape)
            #print(label.shape)
            #print(feats.shape)
            #if torch.isnan(feats).any():
            #    print("⚠️ Tensor 含有 NaN 值！")
            loss = criterion(preds, label)
            #if torch.isnan(loss):
                #print(label)
                #print(feats)
            loss.backward()

            """
            # ----------- 打印梯度長度 (Gradient Norm) -----------
            total_norm = 0.0
            for p in model.parameters():
                if p.grad is not None:
                    # 計算每個參數的二範數 (L2 norm)
                    param_norm = p.grad.data.norm(2)
                    total_norm += param_norm.item() ** 2
            total_norm = total_norm ** 0.5
            print(f"目前 batch 的梯度範數: {total_norm:.4f}")
            # -----------------------------------------------------
            """

            
            optimizer.step()

            

            preds, label = preds.mean(dim=1), label.mean(dim=1)
            correctness_list.append((preds*label>0).float().mean())
            

            total_train_loss += loss.item()
            
        avg_train_loss = total_train_loss / count
        
        train_acc = np.array([t.cpu().numpy() for t in correctness_list]).mean()
        print(f'Train acc : {train_acc}')

        
        # **驗證階段**
        model.eval()
        total_valid_loss = 0.0

        
        correctness_list = []
        with torch.no_grad():  # **關閉梯度，避免影響模型參數**
            for batch in test_loader:
                feats, label = batch
                label = label.unsqueeze(1).repeat(1,30)
                
                feats, label = feats.to(device), label.to(device)

                preds = model(feats) 
                loss = criterion(preds, label)

                preds, label = preds.mean(dim=1), label.mean(dim=1)
                correctness_list.append((preds*label>0).float().mean())
                
                total_valid_loss += loss.item()

        avg_valid_loss = total_valid_loss / count
        
        test_acc = np.array([t.cpu().numpy() for t in correctness_list]).mean()
        print(f'Test acc : {test_acc}')

        # **列印訓練 & 驗證損失**
        print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Valid Loss: {avg_valid_loss:.4f}")
        

train_single_day()


Train acc : 0.47857141494750977
Test acc : 0.5314935445785522
Epoch 1, Train Loss: 4.4690, Valid Loss: 5.7339
Train acc : 0.5035714507102966
Test acc : 0.4743506610393524
Epoch 2, Train Loss: 2.8277, Valid Loss: 4.8904
Train acc : 0.5273810029029846
Test acc : 0.4555194675922394
Epoch 3, Train Loss: 2.8528, Valid Loss: 5.1838
Train acc : 0.4928571581840515
Test acc : 0.4707792103290558
Epoch 4, Train Loss: 2.7881, Valid Loss: 5.1451
Train acc : 0.5404762029647827


KeyboardInterrupt: 

In [ ]:
historical_data = yf.download('AAPL', start='2014-01-01', end='2025-02-01', progress=False)
historical_data["daily_return"] = historical_data["Adj Close"].pct_change()
        
historical_return = historical_data['daily_return']
        date = self.trading_days[idx]


In [ ]:



A = torch.tensor([1,2,3,4,5]).unsqueeze(1)
B = torch.tensor([-1,-2,1,9,9]).unsqueeze(1)
(A*B>0).float().mean()
